In [2]:
import geopandas as gpd  # GeoPandas ist eine Erweiterung von Pandas für die Arbeit mit geospatialen Daten
import matplotlib.pyplot as plt  # Matplotlib wird für die Erstellung von Grafiken und Visualisierungen verwendet
import matplotlib.patches as mpatches  # Matplotlib Patches werden für das Erstellen benutzerdefinierter Legenden und Marker genutzt
from shapely.geometry import Point  # Shapely bietet Werkzeuge zur Erstellung und Analyse von geometrischen Objekten (z. B. Punkten, Linien)
import joblib  # Joblib ermöglicht das Speichern und Laden von Modellen (z. B. DBSCAN-Modellen) in einer komprimierten Form
import numpy as np  # NumPy wird für numerische Berechnungen und die Arbeit mit Arrays verwendet
import pandas as pd  # Pandas ist eine Bibliothek zur Datenmanipulation und -analyse, hauptsächlich mit DataFrames
import fiona  # Fiona wird verwendet, um Geo-Daten (z. B. GeoJSON) zu lesen und zu verarbeiten
from matplotlib.lines import Line2D  # Line2D wird für das Zeichnen von Linien in Matplotlib-Visual

# Lade die ursprünglichen Geo-Koordinaten (erste 45000 Zeilen)
df_coords = pd.read_csv("geo_coords.csv").iloc[:45000]

# Lade die DBSCAN-Labels
dbscan = joblib.load('dbscan_model.pkl')  # Falls das Modell geladen werden muss
cluster_labels = np.load("dbscan_merged_clusters.npy")

# Überprüfen, ob die Längen übereinstimmen
if len(cluster_labels) != len(df_coords):
    raise ValueError("Die Anzahl der Cluster-Labels stimmt nicht mit den Koordinatendaten überein!")

# Füge DBSCAN Cluster-Labels den Daten hinzu
df_coords["Cluster_DBSCAN"] = cluster_labels

# Erstelle ein GeoDataFrame mit den ursprünglichen Koordinaten
geometry = [Point(xy) for xy in zip(df_coords['XGCSWGS84'], df_coords['YGCSWGS84'])]
gdf = gpd.GeoDataFrame(df_coords, geometry=geometry, crs="EPSG:4326")

# Lade die GeoJSON-Datei mit pyogrio als Engine
bez_gdf = gpd.read_file("bezirksgrenzen.geojson", engine='pyogrio')

# Einzigartige Cluster (ohne Rauschen -1)
unique_clusters = sorted(gdf["Cluster_DBSCAN"].unique())

# Visualisierung für jedes Cluster
for cluster in unique_clusters:
    if cluster == -1:
        continue  # Überspringe das Rauschen

    cluster_data = gdf[gdf["Cluster_DBSCAN"] == cluster]

    cluster_center = cluster_data.geometry.unary_union.centroid


    # Erstellen des Plots
    fig, ax = plt.subplots(figsize=(12, 10))

    # Bezirksgrenzen plotten
    bez_gdf.plot(ax=ax, color='white', edgecolor='black', linewidth=0.8, alpha=0.7)

    # Dynamische Farbzuweisung für Cluster
    cluster_color = plt.cm.viridis(cluster / len(unique_clusters))

    # Cluster-Punkte plotten mit einer besseren Farbe 
    cluster_data.plot(ax=ax, color='mediumseagreen', markersize=5, alpha=0.6, label=f"Cluster {cluster}")

    # Clusterzentrum als rotes Kreuz plotten
    ax.plot(cluster_center.x, cluster_center.y, marker='x', color='red', markersize=12, label=f"Zentrum Cluster {cluster}")

    # Legende erstellen

    legend_patches = [
        mpatches.Patch(color='mediumseagreen', label=f"Cluster {cluster}"),
        Line2D([0], [0], color='red', marker='x', linestyle='None', markersize=12, label='Cluster-Zentrum')]
    ax.legend(handles=legend_patches, title="Legende", loc="upper right")


    # Titel setzen
    ax.set_title(f"DBSCAN Cluster {cluster}", fontsize=16)

    # Achsen auf gesamte Karte setzen
    ax.set_xlim([bez_gdf.total_bounds[0], bez_gdf.total_bounds[2]])
    ax.set_ylim([bez_gdf.total_bounds[1], bez_gdf.total_bounds[3]])

    # Speichern der Visualisierung
    plt.savefig(f"output/plots/dbscan_cluster_{cluster}.png", dpi=300, bbox_inches="tight")
    plt.close()

# Speichern der Cluster-Zuordnung
gdf.to_csv("dbscan_cluster_results.csv", index=False)

print("DBSCAN-Cluster-Visualisierung abgeschlossen und Ergebnisse gespeichert.")


DBSCAN-Cluster-Visualisierung abgeschlossen und Ergebnisse gespeichert.
